<a href="https://colab.research.google.com/github/55Riyajain/EDUNET_/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flask flask_sqlalchemy flask_login


In [ ]:
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager

app = Flask(__name__)
app.config['SECRET_KEY'] = 'mysecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///fitness.db'

db = SQLAlchemy(app)
login_manager = LoginManager(app)

from routes import *

if __name__ == "__main__":
    app.run(debug=True)


In [ ]:
from flask_sqlalchemy import SQLAlchemy
from flask_login import UserMixin

db = SQLAlchemy()

class User(db.Model, UserMixin):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    email = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)
    steps = db.Column(db.Integer, default=0)  # Steps count
    calories_burned = db.Column(db.Integer, default=0)  # Calories burned

class ActivityLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)
    exercise_type = db.Column(db.String(100))
    duration = db.Column(db.Integer)  # Duration in minutes
    calories_burned = db.Column(db.Integer)

    user = db.relationship('User', backref=db.backref('activities', lazy=True))


In [ ]:
from flask import render_template, request, redirect, url_for
from app import app, db
from models import User, ActivityLog
from flask_login import login_user, login_required, current_user

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        # Registration logic
        pass
    return render_template('register.html')

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # Login logic
        pass
    return render_template('login.html')

@app.route('/track', methods=['GET', 'POST'])
@login_required
def track_activity():
    if request.method == 'POST':
        exercise = request.form['exercise']
        duration = request.form['duration']
        calories = int(duration) * 10  # Simplified calorie calculation
        new_activity = ActivityLog(user_id=current_user.id, exercise_type=exercise, duration=duration, calories_burned=calories)
        db.session.add(new_activity)
        db.session.commit()

        current_user.steps += int(duration)  # Update steps or other metrics
        current_user.calories_burned += calories
        db.session.commit()

    return render_template('track.html')


In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Fitness Tracker</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <header>
        <h1>Welcome to Fitness Tracker</h1>
        <nav>
            <a href="{{ url_for('login') }}">Login</a> | <a href="{{ url_for('register') }}">Register</a>
        </nav>
    </header>

    <main>
        <h2>Track Your Fitness</h2>
        <form action="{{ url_for('track_activity') }}" method="POST">
            <label for="exercise">Exercise Type:</label>
            <input type="text" id="exercise" name="exercise" required>
            <label for="duration">Duration (minutes):</label>
            <input type="number" id="duration" name="duration" required>
            <button type="submit">Track</button>
        </form>
    </main>
</body>
</html>


In [ ]:
body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
}

header {
    background-color: #333;
    color: white;
    padding: 1rem;
}

header h1 {
    margin: 0;
}

nav a {
    color: white;
    margin-left: 10px;
    text-decoration: none;
}

main {
    padding: 2rem;
}

form {
    margin-top: 20px;
}

label {
    display: block;
    margin: 5px 0;
}

input {
    padding: 5px;
    margin: 5px 0;
}

button {
    padding: 5px 10px;
    background-color: #333;
    color: white;
    border: none;
}


In [ ]:
<script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
<canvas id="myChart" width="400" height="200"></canvas>
<script>
    var ctx = document.getElementById('myChart').getContext('2d');
    var myChart = new Chart(ctx, {
        type: 'line',
        data: {
            labels: ['January', 'February', 'March', 'April', 'May'],
            datasets: [{
                label: 'Calories Burned',
                data: [120, 150, 180, 220, 300],
                borderColor: 'rgba(75, 192, 192, 1)',
                borderWidth: 1
            }]
        }
    });
</script>
